In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

co_df = pd.read_csv("data/co_data.csv")
map_df = pd.read_csv("data/map_data.csv")
user_df = pd.read_csv("data/user_data.csv", index_col= 'userName')
replay_df = pd.read_csv("data/replay_data.csv", index_col= 'replayId')
pick_df = pd.read_csv("data/pick_data.csv", index_col= 'replayId')
ban_df = pd.read_csv("data/ban_data.csv", index_col= 'replayId')

In [25]:
# Only include completed replays
completed_replays = replay_df[replay_df['isBoot'] == False].index

In [27]:
# Calculate amount of matches played and winrate for each map.
# winningPlayerIndex is 1 if p1 wins and 2 if p2 wins. Subtracting these 2, results in p1 win rate.
map_stats_df = replay_df.loc[completed_replays, ['mapName', 'winnerPlayerIndex']].groupby('mapName').apply(lambda x: pd.Series({'count':len(x), 'winRate':(2 - np.mean(x.values))}))
map_stats_df = map_stats_df[map_stats_df['count'] >= 10]

In [58]:
fig = go.Figure(
    go.Bar(
        x= map_stats_df.index,
        y= map_stats_df['winRate'].values,
        customdata= map_stats_df['count'].values,
        marker_color= '#E78F2F',
        texttemplate= "%{y:.2f}",
        hovertemplate= ("P1 Win Rate: %{y:.2f} <br>"
                        "Number of Matches: %{customdata}")
    )
)

fig.update_layout(
    title= ("<b>Player Order Map Bias</b><br>"
            "<i>Maps with at least 10 matches played. Player 1 win rates.</i>")
)
fig.update_xaxes(
    title= "Map Name",
    categoryorder= 'total descending'
)
fig.update_yaxes(
    title= "P1 Win Rate %",
    tickmode= 'array',
    tickvals= [0, 0.25, 0.5, 0.75, 1.0],
)

fig.show()

In [59]:
# fig.write_html("reports/map_winrates.html")